# import Libraries

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time as t
import csv
import random
import threading as th

In [2]:
# Chrome driver Path
chromepath= r"chromedriver.exe"

option= webdriver.ChromeOptions()      # open google chrome session

option.add_argument('-disk-cache-size= 3000000000')   # OPTIONAL,    --->  make memory cache size = 3000000000                                                                                                                                                                                                                                                                                                                         
driver= webdriver.Chrome(executable_path= chromepath, options= option)   # open driver

# ++++++++++++++++++++++++++++++++++++
# We Can Use [ driver.find_element(By.CSS_SELECTOR, "input[type='submit']").text ]  instead of  -->  [ driver.find_element_by_css_selector("input[type='submit']").text ]   to avoid the red Waring which accure while run the code.
titles_lst= []
rating_star_lst= []
rating_lst= []
reviews_lst= []
cpu_lst= []
old_price_lst= []
new_price_lst= []
discount_lst= []
Laptob_link_lst= []


driver.get(f'https://www.flipkart.com/search?q=laptop&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&as-pos=1&as-type=HISTORY&page=1')
nums_of_pages= driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[26]/div/div/span[1]').text
nums_of_pages= int(nums_of_pages.split(' ')[-1])

try:
    pg_counter= 0
    for i in range(1, nums_of_pages+1):
        driver.get(f'https://www.flipkart.com/search?q=laptop&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&as-pos=1&as-type=HISTORY&page={i}')
        showing_pg= driver.find_element_by_css_selector("span[class= '_10Ermr']").text
        splits= showing_pg.split(' ')
        elements_per_pg= int(splits[3]) - int(splits[1]) + 1
        elements_per_pg
        t.sleep(4)

        # scrape Job Titles:
        titles= driver.find_elements_by_css_selector("div[class='_4rR01T']")            
        Titles= [element.text for element in titles]
        titles_lst.extend(Titles)
        t.sleep(2)

        # Scrap Rating Star:
        for j in range(2, elements_per_pg+2):
            rating_satr= driver.find_elements_by_xpath(f"/html/body/div/div/div[3]/div[1]/div[2]/div[{j}]/div/div/div/a/div[2]/div[1]/div[2]/span[1]/div")
            rating_satr2= driver.find_elements_by_xpath(f"/html/body/div/div/div[3]/div[1]/div[2]/div[{j}]/div/div/div/a/div[3]/div[1]/div[2]/span[1]/div")
            ratings_satrs= [element.text for element in rating_satr]
            ratings_satrs2= [element.text for element in rating_satr2]
            rating_star_lst.extend(ratings_satrs)
            rating_star_lst.extend(ratings_satrs2)
        t.sleep(2)

        # Scrap Rating:
        containers= driver.find_elements_by_css_selector("div[class='gUuXy-']")
        for container in containers:
            rating= container.find_element_by_css_selector("span[class='_2_R_DZ']").find_element_by_css_selector("span:nth-child(1)").find_element_by_css_selector("span:nth-child(1)").text.strip()            
            rating_lst.append(rating)
        t.sleep(2)

        # Scrap Reviews:
        containers= driver.find_elements_by_css_selector("div[class='gUuXy-']")
        for container in containers:
            reviews= container.find_element_by_css_selector("span[class='_2_R_DZ']").find_element_by_css_selector("span:nth-child(1)").find_element_by_css_selector("span:nth-child(3)").text.strip()            
            reviews_lst.append(reviews)
        t.sleep(2)

        # Scrap Labtop Charactristics:
        containers= driver.find_elements_by_css_selector("div[class='fMghEO']")       
        for container in containers:       
            cpu= container.find_element_by_css_selector("ul[class='_1xgFaf']").text
            cpu_lst.append(cpu)
        t.sleep(2)

        # Scrap Price Before Discount:
        containers= driver.find_elements_by_css_selector("div[class='_3tbKJL']")
        for container in containers:
            try:              
                old_price= container.find_element_by_css_selector("div[class='_3I9_wc _27UcVY']").text
                old_price_lst.append(old_price)
            except:
                old_price_lst.append("")
        t.sleep(2)

        # Scrap Price After Discount:
        containers= driver.find_elements_by_css_selector("div[class='_3tbKJL']")
        for container in containers:
            try:
                new_price= container.find_element_by_css_selector("div[class='_30jeq3 _1_WHN1']").text
                new_price_lst.append(new_price)
            except:
                new_price_lst.append("")
        t.sleep(2)

        # Scrap Discount Percentage:
        containers= driver.find_elements_by_css_selector("div[class='_3tbKJL']")
        for container in containers:
            try:
                discount= container.find_element_by_css_selector("div[class='_3Ay6Sb']").text
                discount_lst.append(discount)
            except:
                discount_lst.append("")
        t.sleep(2)

        # Scrap Laptops Links:
        containers= driver.find_elements_by_css_selector("div[class='_2kHMtA']")
        for container in containers:
            links= container.find_element_by_css_selector("a[class='_1fQZEK']").get_attribute('href')
            Laptob_link_lst.append(links)
        
        pg_counter += 1
except:
    print(f"We Have Only {pg_counter} pages ")
        
main_lst= [i.split('\n') for i in cpu_lst]

#########################################################


from itertools import zip_longest
from csv import writer

file_list= [titles_lst, rating_star_lst, rating_lst, reviews_lst, old_price_lst, new_price_lst, discount_lst, Laptob_link_lst]
exported= zip_longest(*file_list)

with open(f'Labtops.csv', 'w', encoding="utf-8") as csv_file:
    wr= writer(csv_file)
    wr.writerow(['Title', 'Rating Star', 'Rating', 'Reviews', 'Old Price', 'New Price', 'Discount Percentage', 'Laptop Link'])
    wr.writerows(exported)  


<ipython-input-2-ff2076be65b3>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome(executable_path= chromepath, options= option)   # open driver
C:\Users\nooor\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:451: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\nooor\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:691: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\nooor\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:710: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Pl

We Have Only 41 pages 


In [3]:
len(main_lst)           # Numbers of all laptops in all pages

984

In [4]:
main_lst= [i.split('\n') for i in cpu_lst]
main_lst

[['AMD Ryzen 3 Dual Core Processor',
  '8 GB DDR4 RAM',
  '64 bit Windows 11 Operating System',
  '512 GB SSD',
  '39.62 cm (15.6 Inch) Display',
  '1 Year Onsite Warranty'],
 ['AMD Ryzen 5 Quad Core Processor',
  '8 GB DDR4 RAM',
  '64 bit Windows 11 Operating System',
  '512 GB SSD',
  '39.62 cm (15.6 Inch) Display',
  '1 Year Onsite Warranty'],
 ['Intel Core i7 Processor (10th Gen)',
  '16 GB DDR4 RAM',
  '64 bit Windows 10 Operating System',
  '512 GB SSD',
  '39.62 cm (15.6 inch) Display',
  'Cooler Boost 3, True Color 2.0, Nahimic 3, Creator Center',
  '1 Year Warranty'],
 ['AMD Ryzen 3 Quad Core Processor',
  '8 GB DDR4 RAM',
  '64 bit Windows 11 Operating System',
  '512 GB SSD',
  '35.56 cm (14 Inch) Display',
  '1 Year onsite warranty'],
 ['AMD Ryzen 7 Quad Core Processor',
  '16 GB DDR4 RAM',
  '64 bit Windows 11 Operating System',
  '512 GB SSD',
  '39.62 cm (15.6 inch) Display',
  'Windows 11 Home, Microsoft Office Home & Student 2021, 1 Year Mcafee',
  '1 Year On site Man

In [5]:
import pandas as pd

df1= pd.read_csv('Labtops.csv')
print(df1.shape)      # 199 jobs, 7 info about jobs

with pd.option_context('display.max_colwidth', None):
    display(df1.head())

(984, 8)


,Title,Rating Star,Rating,Reviews,Old Price,New Price,Discount Percentage,Laptop Link
0,Lenovo IdeaPad 1 Ryzen 3 Dual Core 3250U - (8 GB/512 GB SSD/Windows 11 Home) 15ADA7 Thin and Light Lap...,4.3,140 Ratings,31 Reviews,"₹54,490","₹35,990",33% off,https://www.flipkart.com/lenovo-ideapad-1-ryzen-3-dual-core-3250u-8-gb-512-gb-ssd-windows-11-home-15ada7-thin-light-laptop/p/itm3d18297fc081a?pid=COMGFM5BYHQPXCRZ&lid=LSTCOMGFM5BYHQPXCRZHYWG3P&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_1&otracker=search&otracker1=search&iid=en_2QFTLY9SHLJH8E8Z%2BmLH5OTzx0giwS%2BklXFEjGtgZGqzTPiAPLngT5eU3qHCgOisQ7z6Yvxla%2Fl7lMO%2Fm143SQ%3D%3D&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09
1,Lenovo IdeaPad 1 Ryzen 5 Quad Core 3500U - (8 GB/512 GB SSD/Windows 11 Home) 15ADA7 Thin and Light Lap...,4.4,555 Ratings,82 Reviews,"₹60,090","₹42,990",28% off,https://www.flipkart.com/lenovo-ideapad-1-ryzen-5-quad-core-3500u-8-gb-512-gb-ssd-windows-11-home-15ada7-thin-light-laptop/p/itm6253939cee63b?pid=COMGFM5BXGAMEWPE&lid=LSTCOMGFM5BXGAMEWPESNT1TZ&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_2&otracker=search&otracker1=search&iid=en_2QFTLY9SHLJH8E8Z%2BmLH5OTzx0giwS%2BklXFEjGtgZGqL3UJ7Adlv2OF72P486t1f2956bCf7NdMu58cBoUmZEA%3D%3D&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09
2,MSI Prestige 15 Core i7 10th Gen - (16 GB/512 GB SSD/Windows 10 Home/4 GB Graphics) Prestige 15 A10SC-...,4.4,17 Ratings,2 Reviews,"₹1,34,990","₹1,13,490",15% off,https://www.flipkart.com/msi-prestige-15-core-i7-10th-gen-16-gb-512-gb-ssd-windows-10-home-4-graphics-a10sc-239in-thin-light-laptop/p/itm070ca9f1628ef?pid=COMFMJDVKEMKN4HE&lid=LSTCOMFMJDVKEMKN4HELPHXUF&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_3&otracker=search&otracker1=search&iid=ff74c658-8ebc-48ad-8f2b-6085cebd76a7.COMFMJDVKEMKN4HE.SEARCH&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09
3,HP Ryzen 3 Quad Core 5300U - (8 GB/512 GB SSD/Windows 11 Home) 14s-fq1089au Thin and Light Laptop,4.3,"1,696 Ratings",197 Reviews,"₹52,900","₹38,890",26% off,https://www.flipkart.com/hp-ryzen-3-quad-core-5300u-8-gb-512-gb-ssd-windows-11-home-14s-fq1089au-thin-light-laptop/p/itm30e084d034979?pid=COMGAY9Y7WXZYFDA&lid=LSTCOMGAY9Y7WXZYFDALA89IW&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_4&otracker=search&otracker1=search&iid=ff74c658-8ebc-48ad-8f2b-6085cebd76a7.COMGAY9Y7WXZYFDA.SEARCH&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09
4,ASUS Vivobook 15 Ryzen 7 Quad Core AMD R7-3700U - (16 GB/512 GB SSD/Windows 11 Home) M515DA-BQ722WS La...,4.3,263 Ratings,23 Reviews,"₹74,990","₹44,990",40% off,https://www.flipkart.com/asus-vivobook-15-ryzen-7-quad-core-amd-r7-3700u-16-gb-512-gb-ssd-windows-11-home-m515da-bq722ws-laptop/p/itm5c77300ec1eb8?pid=COMGCRFYZF5TK7ZU&lid=LSTCOMGCRFYZF5TK7ZU4THGJF&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_5&otracker=search&otracker1=search&iid=ff74c658-8ebc-48ad-8f2b-6085cebd76a7.COMGCRFYZF5TK7ZU.SEARCH&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09


In [24]:
lst= [i.split('\n') for i in cpu_lst]
llsstt= []
for l in lst:
    llsstt.append(len(l))

import numpy as np 
np.max(llsstt)           

12

In [25]:
set(llsstt)

{5, 6, 7, 8, 9, 10, 11, 12}

In [32]:
print(len( [ i for i in llsstt if i == 5] ))
print(len( [ i for i in llsstt if i == 6] ))
print(len( [ i for i in llsstt if i == 7] ))
print(len( [ i for i in llsstt if i == 8] ))
print(len( [ i for i in llsstt if i == 9] ))
print(len( [ i for i in llsstt if i == 10] ))
print(len( [ i for i in llsstt if i == 11] ))
print(len( [ i for i in llsstt if i == 12] ))        # So we will take which len is [5 , 6 , 7] because has a lot of data , and remove which len is [8, 9, 10, 11, 12] because has a little of data.

11
444
503
13
1
3
5
4


In [33]:
11 + 444 + 503 + 13 + 1 + 3 + 5 + 4           # if we add all data of all len we will have 984 which is the number of all laptops in all pages.

984

In [36]:
# So we will take which len is [5 , 6 , 7] because has a lot of data , and remove which len is [8, 9, 10, 11, 12] because has a little of data: 

lst= [i.split('\n') for i in cpu_lst]
filtered_lst= [i for i in lst if len(i) == 5 or len(i) == 6 or len(i) == 7]

In [37]:
import pandas as pd

df2= pd.DataFrame(filtered_lst, columns= ['CPU', 'RAM', 'Operating System', 'Hard Disk', 'Screen size', 'Usage', 'Warranty'])
df2.head()

,CPU,RAM,Operating System,Hard Disk,Screen size,Usage,Warranty
0,AMD Ryzen 3 Dual Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,39.62 cm (15.6 Inch) Display,1 Year Onsite Warranty,None
1,AMD Ryzen 5 Quad Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,39.62 cm (15.6 Inch) Display,1 Year Onsite Warranty,None
2,Intel Core i7 Processor (10th Gen),16 GB DDR4 RAM,64 bit Windows 10 Operating System,512 GB SSD,39.62 cm (15.6 inch) Display,"Cooler Boost 3, True Color 2.0, Nahimic 3, Cre...",1 Year Warranty
3,AMD Ryzen 3 Quad Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,35.56 cm (14 Inch) Display,1 Year onsite warranty,None
4,AMD Ryzen 7 Quad Core Processor,16 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,39.62 cm (15.6 inch) Display,"Windows 11 Home, Microsoft Office Home & Stude...",1 Year On site Manufacturing warranty


In [38]:
all_df= pd.concat([df1, df2], axis= 1)

with pd.option_context('display.max_colwidth', None):
    display(all_df)

,Title,Rating Star,Rating,Reviews,Old Price,New Price,Discount Percentage,Laptop Link,CPU,RAM,Operating System,Hard Disk,Screen size,Usage,Warranty
0,Lenovo IdeaPad 1 Ryzen 3 Dual Core 3250U - (8 GB/512 GB SSD/Windows 11 Home) 15ADA7 Thin and Light Lap...,4.3,140 Ratings,31 Reviews,"₹54,490","₹35,990",33% off,https://www.flipkart.com/lenovo-ideapad-1-ryzen-3-dual-core-3250u-8-gb-512-gb-ssd-windows-11-home-15ada7-thin-light-laptop/p/itm3d18297fc081a?pid=COMGFM5BYHQPXCRZ&lid=LSTCOMGFM5BYHQPXCRZHYWG3P&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_1&otracker=search&otracker1=search&iid=en_2QFTLY9SHLJH8E8Z%2BmLH5OTzx0giwS%2BklXFEjGtgZGqzTPiAPLngT5eU3qHCgOisQ7z6Yvxla%2Fl7lMO%2Fm143SQ%3D%3D&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09,AMD Ryzen 3 Dual Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,39.62 cm (15.6 Inch) Display,1 Year Onsite Warranty,None
1,Lenovo IdeaPad 1 Ryzen 5 Quad Core 3500U - (8 GB/512 GB SSD/Windows 11 Home) 15ADA7 Thin and Light Lap...,4.4,555 Ratings,82 Reviews,"₹60,090","₹42,990",28% off,https://www.flipkart.com/lenovo-ideapad-1-ryzen-5-quad-core-3500u-8-gb-512-gb-ssd-windows-11-home-15ada7-thin-light-laptop/p/itm6253939cee63b?pid=COMGFM5BXGAMEWPE&lid=LSTCOMGFM5BXGAMEWPESNT1TZ&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_2&otracker=search&otracker1=search&iid=en_2QFTLY9SHLJH8E8Z%2BmLH5OTzx0giwS%2BklXFEjGtgZGqL3UJ7Adlv2OF72P486t1f2956bCf7NdMu58cBoUmZEA%3D%3D&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09,AMD Ryzen 5 Quad Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,39.62 cm (15.6 Inch) Display,1 Year Onsite Warranty,None
2,MSI Prestige 15 Core i7 10th Gen - (16 GB/512 GB SSD/Windows 10 Home/4 GB Graphics) Prestige 15 A10SC-...,4.4,17 Ratings,2 Reviews,"₹1,34,990","₹1,13,490",15% off,https://www.flipkart.com/msi-prestige-15-core-i7-10th-gen-16-gb-512-gb-ssd-windows-10-home-4-graphics-a10sc-239in-thin-light-laptop/p/itm070ca9f1628ef?pid=COMFMJDVKEMKN4HE&lid=LSTCOMFMJDVKEMKN4HELPHXUF&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_3&otracker=search&otracker1=search&iid=ff74c658-8ebc-48ad-8f2b-6085cebd76a7.COMFMJDVKEMKN4HE.SEARCH&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09,Intel Core i7 Processor (10th Gen),16 GB DDR4 RAM,64 bit Windows 10 Operating System,512 GB SSD,39.62 cm (15.6 inch) Display,"Cooler Boost 3, True Color 2.0, Nahimic 3, Creator Center",1 Year Warranty
3,HP Ryzen 3 Quad Core 5300U - (8 GB/512 GB SSD/Windows 11 Home) 14s-fq1089au Thin and Light Laptop,4.3,"1,696 Ratings",197 Reviews,"₹52,900","₹38,890",26% off,https://www.flipkart.com/hp-ryzen-3-quad-core-5300u-8-gb-512-gb-ssd-windows-11-home-14s-fq1089au-thin-light-laptop/p/itm30e084d034979?pid=COMGAY9Y7WXZYFDA&lid=LSTCOMGAY9Y7WXZYFDALA89IW&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_4&otracker=search&otracker1=search&iid=ff74c658-8ebc-48ad-8f2b-6085cebd76a7.COMGAY9Y7WXZYFDA.SEARCH&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09,AMD Ryzen 3 Quad Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,35.56 cm (14 Inch) Display,1 Year onsite warranty,None
4,ASUS Vivobook 15 Ryzen 7 Quad Core AMD R7-3700U - (16 GB/512 GB SSD/Windows 11 Home) M515DA-BQ722WS La...,4.3,263 Ratings,23 Reviews,"₹74,990","₹44,990",40% off,https://www.flipkart.com/asus-vivobook-15-ryzen-7-quad-core-amd-r7-3700u-16-gb-512-gb-ssd-windows-11-home-m515da-bq722ws-laptop/p/itm5c77300ec1eb8?pid=COMGCRFYZF5TK7ZU&lid=LSTCOMGCRFYZF5TK7ZU4THGJF&marketplace=FLIPKART&q=laptop&store=6bo%2Fb5g&srno=s_1_5&otracker=search&otracker1=search&iid=ff74c658-8ebc-48ad-8f2b-6085cebd76a7.COMGCRFYZF5TK7ZU.SEARCH&ssid=02zqarbp6o0000001658444100894&qH=312f91285e048e09,AMD Ryzen 7 Quad Core Processor,16 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,39.62 cm (15.6 inch) Display,"Windows 11 Home, Microsoft Office Home & Student 2021, 1 Year Mcafee",1 Year On site Manufacturing warranty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,..